# Como um Agent é construído

A ideia central dos agentes é usar um modelo de linguagem para escolher uma sequência de ações a serem executadas. Em cadeias, uma sequência de ações é codificada diretamente (no código). Em agentes, um modelo de linguagem é usado como um motor de raciocínio para determinar quais ações devem ser tomadas e em qual ordem.

## Criando as tools que usaremos

In [18]:
import requests
import datetime

from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field

import wikipedia
wikipedia.set_lang('pt')

class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')


@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))

        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return f'{temp_atual}ºC'
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

## Revisando a utilização das tools

In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])
chat = ChatOpenAI()

tools = [busca_wikipedia, retorna_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)

In [20]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [21]:
resposta = chain.invoke({'input': 'Qual a temperatura em Floripa?'})
resposta

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -27.5953787, 'longitude': -48.5480499}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -27.5953787, 'longitude': -48.5480499}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5953787,"longitude":-48.5480499}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 153, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9ee389cd-e54e-477d-b466-abee4c4081be-0')])

In [22]:
resposta.tool

'retorna_temperatura_atual'

In [23]:
resposta.tool_input

{'latitude': -27.5953787, 'longitude': -48.5480499}

In [24]:
resposta.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5953787,"longitude":-48.5480499}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 153, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9ee389cd-e54e-477d-b466-abee4c4081be-0')]

## Adicionando o racicínio do agent as mensagens (agent_scretchpad)

Temos que adicionar junto as nossas mensagens um campo que armazenará o raciocínio atual do modelo chamado agent_scratchpad. Para isso, utilizamos um MessagesPlaceholder ao nosso prompt. Ele guardará espaço para o raciocínio e, caso o modelo não esteja gerando um raciocínio no momento, o MessagesPlaceholder não é utilizado.

In [25]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [26]:
resposta_inicial = chain.invoke({
    'input': 'Qual a temperatura em Floripa?',
    'agent_scratchpad': []})
resposta_inicial

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -27.5954, 'longitude': -48.548}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -27.5954, 'longitude': -48.548}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 153, 'total_tokens': 180}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-722c78ea-a704-41e2-9205-564c3e8270ab-0')])

In [27]:
observacao = tool_run[resposta_inicial.tool].run(resposta_inicial.tool_input)
observacao

'21.9ºC'

Podemos utilizar a função format_to_openai_function_messages para modificar o formato da resposta de forma que ela possa ser enviada, junto da observação, de volta ao modelo. No caso o que está ocorrendo é o modelo está pedindo que uma tool seja rodada, estamos rodando a tool e gerando uma observação, e a gora enviamos novamente para o modelo a pergunta original, a mensagem do próprio modelo dizendo que precisava que a tool fosse rodada e a observação gerada pela ferramenta.

In [28]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages

format_to_openai_function_messages([(resposta_inicial, observacao)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 153, 'total_tokens': 180}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-722c78ea-a704-41e2-9205-564c3e8270ab-0'),
 FunctionMessage(content='21.9ºC', name='retorna_temperatura_atual')]

In [29]:
resposta_final = chain.invoke({
    'input': 'Qual a temperatura em Floripa?',
    'agent_scratchpad': format_to_openai_function_messages([(resposta_inicial, observacao)])})
resposta_final

AgentFinish(return_values={'output': 'A temperatura em Florianópolis é de 21.9ºC no momento.'}, log='A temperatura em Florianópolis é de 21.9ºC no momento.')

### Criando um loop de racicínio

Por fim, podemos criar um loop que adiciona a automaticamente as chamadas de função e observações e fica chamando o modelo novamente até que a mensagem de AgentFinish seja recebida.

In [31]:
from langchain.schema.agent import AgentFinish

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'agent_scratchpad': format_to_openai_function_messages(passos_intermediarios)
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))


In [32]:
run_agent('Qual é a temperatura de Floripa?')

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 21.5ºC.'}, log='A temperatura atual em Florianópolis é de 21.5ºC.')

Modificamos um pouco o formato para padronizar ao funcionamento do LangChain para agents

In [38]:
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough


pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)
chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))


In [39]:
pass_through.invoke({'input': 'Qual é a temperatura de Floripa?', 'intermediate_steps': []})

{'input': 'Qual é a temperatura de Floripa?',
 'intermediate_steps': [],
 'agent_scratchpad': []}

In [40]:
run_agent('Qual é a temperatura de Floripa?')

AgentFinish(return_values={'output': 'A temperatura atual em Floripa é de 21.5ºC.'}, log='A temperatura atual em Floripa é de 21.5ºC.')

## O que temos no final?

### Um Agent

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)
chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

### Um AgentExecutor

In [ ]:
def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))